In [167]:
import pymongo, re, json
from collections import defaultdict

In [168]:
key_words = ['a', 'able', 'about', 'above', 'across', 'actually', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', "aren't", 'as', 'aside', 'at', 'available', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'being', 'below', 'beside', 'besides', 'between', 'both', 'but', 'by', "can't", 'cannot', 'certain', 'certainly', 'clearly', 'co', 'com', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', "couldn't", 'course', 'currently', 'definitely', 'described', 'despite', 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'downwards', 'during', 'each', 'else', 'elsewhere', 'enough', 'especially', 'even', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'far', 'few', 'following', 'for', 'former', 'formerly', 'forth', 'from', 'further', 'furthermore', 'get', 'gets', 'getting', 'go', 'goes', 'going', 'gone', 'got', 'gotten', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'hence', 'her', 'here', "here's", 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', "how's", 'howbeit', 'i', "i'd", "i'll", "i'm", "i've", 'ie', 'if', 'ignored', 'immediate', 'in', 'inasmuch', 'indicate', 'indicated', 'indicates', 'inner', 'insofar', 'instead', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', 'keep', 'keeps', 'kept', 'know', 'known', 'knows', 'last', 'latter', 'latterly', 'least', 'lest', "let's", 'like', 'liked', 'likely', 'likewise', 'look', 'looking', 'looks', 'ltd', 'may', 'maybe', 'me', 'meanwhile', 'merely', 'might', 'more', 'moreover', 'most', 'mostly', 'much', 'must', "mustn't", 'my', 'myself', "n't", 'near', 'nearly', 'necessary', 'need', 'needs', 'neither', 'never', 'nevertheless', 'next', 'no', 'non', 'none', 'nor', 'normally', 'not', 'nothing', 'novel', 'now', 'nowhere', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'particular', 'particularly', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provides', 'que', 'quite', 'qv', 'rather', 'rd', 're', 'really', 'reasonably', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'several', 'shall', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'since', 'so', 'some', 'somebody', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', "t's", 'take', 'taken', 'taking', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', "that's", 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', "there's", 'thereafter', 'thereby', 'therefore', 'therein', 'theres', 'thereupon', 'these', 'they', "they'd", "they'll", "they're", "they've", 'think', 'third', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thus', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'twice', 'un', 'under', 'unfortunately', 'unless', 'unlikely', 'until', 'unto', 'up', 'upon', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'value', 'various', 'very', 'via', 'viz', 'vs', 'want', 'wants', 'was', "wasn't", 'way', 'we', "we'd", "we'll", "we're", "we've", 'welcome', 'well', 'went', 'were', "weren't", 'what', "what's", 'whatever', 'when', "when's", 'whence', 'whenever', 'where', "where's", 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', "who's", 'whoever', 'whole', 'whom', 'whose', 'why', "why's", 'will', 'willing', 'wish', 'with', "won't", 'wonder', 'would', "wouldn't", 'yes', 'yet', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

punctuation_pattern = r'[!\"#$%&\'()*+,-./:;<=>?@\[\]\^_`{|}~—]'

In [169]:
def checkKeyWords(string: str) -> bool:
    """Checks against the keywords list for any matches in the string input

    Args:
        string (str): String that will be checked

    Returns:
        bool: True if the string does not coincide with the list, False if there is a match
    """

    for keyword in key_words:
        match = re.search(rf'^{keyword}$', string=string)
        if(match):
            return False
    
    return True

def processInitialString(queryRaw: str):
    """Function that takes the initial string query and parses it to a list with removed punctuation signs and common words

    Args:
        queryRaw (str): Initial query input string

    Returns:
        _type_: List of strings from initial query parsed
    """

    query_list = queryRaw.lower().split(' ')
    query_list_processed = []

    for query in query_list:

        query_processed = re.sub(punctuation_pattern, '', query)
        if checkKeyWords(query_processed): query_list_processed.append(query_processed)

    return query_list_processed

In [170]:
def checkStringDescriptions(hts_document: list[str,any], query_string: str, count: int) -> dict[str,any]:
    """Function that goes through the hts document gathered from the db and gets the relevant records with their index and hts number if present and returns them in a dictionary

    Args:
        hts_document (list[str,any]): Hts document gathered from DB
        query_string (str): String that will be searched against Hts document
        count (int): Count of occurrences of the string in the Hts document

    Returns:
        dict[str,any]: Resulting dictionary with chapter header, count and Hts document htsno's and indexes
    """

    result = {
        'chap': hts_document['header'],
        'count': count,
        'data': []
    }
    for key, doc in enumerate(hts_document['data']):
        
        description = re.sub(punctuation_pattern, '', doc['description'].lower())
        if re.search(query_string, description):
            
            result['data'].append({
                'htsno': doc.get('htsno', None),
                'indexHTS': key
            })
    
    return result

In [171]:
class Connection:
    """Class that connects to the database creating all necessary methods for connection and closing connection, as well as the base database for adding new HTS records and the string_dict collection too
    """

    def __init__(self):
        """_init_ function of the class, defines the connection variables

        Args:
            db_path (str): Path to the database connection on MongoDB
        """

        db_path = 'localhost'
        db_port = 27017

        try:
            self.client = pymongo.MongoClient(host= db_path, port= db_port)
            self.db = self.client['hts']
            self.collection_records = self.db['hts_records']
            self.collection_string_dict = self.db['string_dict']
            print(f'Connected to {db_path} on {db_port}')


        except Exception as exception:
            print('Error connecting to database')
            print(exception)
            raise

    def closeConnection(self):
        """Close connection function, closes current connection created in the Connection class
        """
        try:
            self.client.close()
            print('Closed connection to mongodb')
        except Exception as exception:
            print('Error closing connection')
            print(exception)
    
    def queryRecordsHTS(self, query_groups: list[dict[str, any]]) -> list[dict[str,any]]:
        """This function queries the hts_records collection to retrieve the full records for a given list of queries (hts numbers)

        Args:
            query_groups (list[dict[str, any]]): Query hts groups organized for database query.

        Returns:
            list[dict[str,any]]: Resulting records from DB in list.
        """

        result = []

        for group in query_groups:

            try:
                document = self.collection_records.find_one(
                    {'header': group['main_group']}
                )
                if document == None: 
                    result.append('No result')
                    print(f"Warning, no result found for {group['main_group']}")
                else: result.append(document)
            except Exception as exception:
                print(f"Failed query of record {group['main_group']}")
                print(exception)
        
        return result
    
    def queryStringDict(self, queryList: list[str]) -> dict[str,any]:
        """This function queries the string_dict collection in the DB and returns a list of results based on the query list provided of strings to query

        Args:
            queryList (list[str]): List of strings to pass to query

        Returns:
            list[dict[str,any]]: List of documents retrieved from the database
        """

        def grabChapterInfo(connection: Connection, document: dict[str,any]) -> dict[str,any]:
            """Function that queries the hts record collection and grabs the relevant record information for each of the chapters a string is present in the query

            Args:
                connection (Connection): Connection object from the database
                document (dict[str,any]): Document with the string_dict record information

            Returns:
                dict[str,any]: Returns a complete object with the chapter header information, the count of repeated times the string is present and the HTS record data
            """
            chapters = []

            for data in document['chaps']:
                chapter = connection.collection_records.find_one({'_id': data['chap']})
                chapters.append(checkStringDescriptions(chapter, document['string'], data['count']))
            
            return chapters



        result = {}

        for query in queryList:
            try:
                document = self.collection_string_dict.find_one({'string': query})
                if document != None:
                    result[query] = grabChapterInfo(self, document)
                
            except Exception as exception:
                print(f'Failed to query string: {query}')
                print(exception)

        return result

In [172]:
def mapStringQuery(query_result: dict[list[dict[str,any]]]) -> dict[list[dict[str,any]]]:
    """Function that works by generating a map of the strings repeated in specific subchapters for better query occurrence handling.

    Args:
        query_result (dict[list[dict[str,any]]]): Initial query result of strings and their corresponding chapter information

    Returns:
        dict[list[dict[str,any]]]: Mapped result containing the information on each main chapter and the max occurrence of each string and string combinations found, as well as HTS number and index information for further query result structuring
    """

    chapter_classification = defaultdict(list)

    for query in query_result:

        for chap_item in query_result[query]:

            chapter_classification[chap_item['chap']].append({
                'query': query ,
                'count': chap_item['count'],
                'data': chap_item['data']
            })

        #This sorts by number of strings in the chapter
        sorted_map_str = dict(sorted(chapter_classification.items(), key=lambda item: len(item[1]), reverse=True))

        mapped_result_occurrence = {}
        #This sorts if two or more strings appear in the same indexHTS and creates a key of max_coincidence
        for chap in sorted_map_str:
            hts_sub_chap = {}
            for item in sorted_map_str[chap]:
                query_str = item['query']
                for data in item['data']:
                    if data['htsno'] == None:
                        key_EH = f"INDX-{data['indexHTS']}"
                        if key_EH in hts_sub_chap:
                            hts_sub_chap[key_EH]['ocurrence'] += 1
                            hts_sub_chap[key_EH]['query_str'].append(query_str)
                        else:
                            hts_sub_chap[key_EH] = {
                                'ocurrence': 1,
                                'htsdata':data,
                                'query_str': [query_str]
                            }
                    elif data['htsno'] in hts_sub_chap: 
                        hts_sub_chap[data['htsno']]['ocurrence'] += 1
                        hts_sub_chap[data['htsno']]['query_str'].append(query_str)
                    else: hts_sub_chap[data['htsno']] = {
                        'ocurrence': 1,
                        'htsdata':data,
                        'query_str': [query_str]
                    }
            
            mapped_result_occurrence[chap] = hts_sub_chap

    return mapped_result_occurrence

def sortMapByOccurrence(mapped_result_occurrence: dict[str,dict[str,any]]) -> dict[str,dict[str,any]]:
    """Function that sorts the mapped_result_ocurrence dictionary by both the number of ocurrences of queries, and then by the number of sub_chapters that were matched by the queries

    Args:
        mapped_result_occurrence (dict[str,dict[str,any]]): Mapped object with sorted_result information of sub_chapters, max_ocurrence and queries list

    Returns:
        dict[str,dict[str,any]]: Sorted object with order primarily by max_ocurrence and secondarily by sub_chapters
    """
    sorted_result = defaultdict(dict)
    for key, value in mapped_result_occurrence.items():

        ocurrence = 0
        queries = []
        for sub_key in value.keys(): 
            ocurrence  = ocurrence if ocurrence > value[sub_key]['ocurrence'] else value[sub_key]['ocurrence']
            queries.extend(value[sub_key]['query_str'])
            queries = list(set(queries))
        
        sorted_result[key]['data'] = dict(sorted(value.items(), key= lambda item: -item[1]['ocurrence']))
        sorted_result[key]['sorted_result'] = {
            'sub_chapters': len(value),
            'max_ocurrence': ocurrence,
            'queries': queries
        }
    
    sorted_result = dict(sorted(sorted_result.items(), key= lambda item: (-item[1]['sorted_result']['max_ocurrence'], -item[1]['sorted_result']['sub_chapters'])))

    return sorted_result

In [173]:
queryList = processInitialString("textile components cotton")
print(queryList)

['textile', 'components', 'cotton']


In [174]:
connection = Connection()
step_one_result = connection.queryStringDict(queryList)
connection.closeConnection()
mapped_result = mapStringQuery(step_one_result)
sorted_result = sortMapByOccurrence(mapped_result)

Connected to localhost on 27017
Closed connection to mongodb


In [175]:
with open('testing_step_one_result.json', 'w') as file:
    json.dump(step_one_result, file)

with open('testing_mapped_result.json', 'w') as file:
    json.dump(mapped_result, file)

with open('testing_sorted_result.json', 'w') as file:
    json.dump(sorted_result, file)